In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Aykhal,RU,2022-01-31 23:57:16,66.0000,111.5000,-27.47,100,89,4.97
1,1,Atuona,PF,2022-01-31 23:57:25,-9.8000,-139.0333,78.46,77,14,18.12
2,2,Bluff,NZ,2022-01-31 23:57:38,-46.6000,168.3333,75.79,80,37,3.31
3,3,Busselton,AU,2022-01-31 23:57:39,-33.6500,115.3333,64.04,53,1,17.31
4,4,Kruisfontein,ZA,2022-01-31 23:57:39,-34.0033,24.7314,70.88,98,99,4.63


In [3]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=temps)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 80
What is the maximum temperature you would like for your trip? 95


In [13]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
12,12,Nakhon Nayok,TH,2022-01-31 23:57:44,14.1667,101.1000,82.02,88,16,3.89
16,16,Vaini,TO,2022-01-31 23:57:46,-21.2000,-175.2000,86.16,79,75,10.36
33,33,Bowen,AU,2022-01-31 23:57:53,-20.0167,148.2333,84.42,71,10,4.16
42,42,Hobart,AU,2022-01-31 23:57:58,-42.8794,147.3294,83.05,52,40,5.01
68,68,Butaritari,KI,2022-01-31 23:54:56,3.0707,172.7902,81.45,79,78,13.31
74,74,Kloulklubed,PW,2022-01-31 23:59:10,7.0419,134.2556,82.98,73,100,18.37
82,82,Koungou,YT,2022-01-31 23:57:02,-12.7336,45.2042,82.11,83,0,8.05
89,89,Kavaratti,IN,2022-01-31 23:55:03,10.5669,72.6420,80.46,72,2,10.54
100,100,Thinadhoo,MV,2022-02-01 00:00:29,0.5333,72.9333,82.56,69,19,11.05
108,108,Hithadhoo,MV,2022-02-01 00:00:33,-0.6000,73.0833,82.11,75,26,11.68


In [14]:
preferred_cities_df.count()

City_ID       68
City          68
Country       68
Date          68
Lat           68
Lng           68
Max Temp      68
Humidity      68
Cloudiness    68
Wind Speed    68
dtype: int64

In [15]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
12,Nakhon Nayok,TH,82.02,14.1667,101.1000,
16,Vaini,TO,86.16,-21.2000,-175.2000,
33,Bowen,AU,84.42,-20.0167,148.2333,
42,Hobart,AU,83.05,-42.8794,147.3294,
68,Butaritari,KI,81.45,3.0707,172.7902,
74,Kloulklubed,PW,82.98,7.0419,134.2556,
82,Koungou,YT,82.11,-12.7336,45.2042,
89,Kavaratti,IN,80.46,10.5669,72.6420,
100,Thinadhoo,MV,82.56,0.5333,72.9333,
108,Hithadhoo,MV,82.11,-0.6000,73.0833,


In [18]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
   # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.


In [19]:
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
12,Nakhon Nayok,TH,82.02,14.1667,101.1000,ห้างหุ้นส่วนจำกัด เอสพีรีสอร์ท
16,Vaini,TO,86.16,-21.2000,-175.2000,Keleti Beach Resort
33,Bowen,AU,84.42,-20.0167,148.2333,Castle Motor Lodge Motel
42,Hobart,AU,83.05,-42.8794,147.3294,Mantra on Collins Hobart
68,Butaritari,KI,81.45,3.0707,172.7902,Isles Sunset Lodge
...,...,...,...,...,...,...
525,Catamarca,AR,84.78,-28.4696,-65.7852,Amerian Catamarca Park Hotel
532,Tautira,PF,86.02,-17.7333,-149.1500,Pension Oaoa
552,Biak,ID,80.44,-0.9131,122.8766,Penginapan Kim
555,Kulhudhuffushi,MV,81.64,6.6221,73.0700,Haajy Guest House 1


In [23]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [24]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [25]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{column1}</dd>
<dt>Another name</dt><dd>{column2}</dd>
</dl>
"""

In [26]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [27]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [28]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))